# Martech - Query Data

## Initialise Connection

In [31]:
from astra_db import AstraSession

session = AstraSession().session

## Define Prepared Statements

### PROFILECAP

In [32]:
# PROFILECAP
get_profile_cap_stmt = session.prepare("SELECT sum(count) from profile_cap WHERE customer_id=? and channel=? AND comm_date_bucket > ? AND comm_date_bucket < ?")

### ACTYCAP

In [33]:
# ACTYCAP

get_acty_cap_stmt = session.prepare("SELECT sum(count) from acty_cap WHERE activity_name=? AND comm_date_bucket > ? AND comm_date_bucket < ?")

### PRTYCAP

In [34]:
# PRTYCAP
get_prty_cap_stmt = session.prepare("SELECT sum(count) from prty_cap WHERE category_group = ? AND category = ? AND comm_date_bucket > ? AND comm_date_bucket < ?")

### CHANNELCAP

In [35]:
# CHANNELCAP
get_channel_cap_stmt = session.prepare("SELECT sum(count) from channel_cap WHERE channel=? AND comm_date_bucket > ? AND comm_date_bucket < ?")

## Define Entities

In [37]:
# Define Entities

# customer_id
customer_id_array = [
    '0b6ec30f-8b1a-406f-82f6-89222a017449',
    '8435a9b9-9e88-495c-a297-dfd25d44428b',
    '8e12d570-e47d-43b9-9435-b2020899860b',
    '9e53cd14-0316-46ff-b913-30228fcc5b73',
    'a6e198e0-674d-4a27-ba01-595a78fda2ef',
    'af9e65bd-4474-40c9-8d16-7b82035d3e0f',
    'b191cef3-4624-4e88-9a93-73de506e4be7',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
    'cf3a5d68-e66f-4ee7-a2d1-2666c5366540',
    'f68e8bc9-86b7-4c8e-ba74-87d31d27a531',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
]

# channel
channel_array = ['email', 'sms', 'push', 'in-app', 'direct']

# category_group
category_group_array = ['ctgy_grp_1', 'ctgy_grp_2', 'ctgy_grp_3', 'ctgy_grp_4', 'ctgy_grp_5']

# category
category_array = ['ctgy_1', 'ctgy_2', 'ctgy_3', 'ctgy_4', 'ctgy_5']

# activity_name
activity_name_array = ['credit_card', 'mortgage', 'business', 'retail', 'investment']

## Query Data

In [38]:
start_date = '2024120100'
end_date = '2024123123'

### PROFILECAP

In [43]:
# comm_date_bucket - yyyymmddhh - 2024120100
rows = session.execute(get_profile_cap_stmt, [customer_id_array[0], 'email', start_date, end_date])
print(rows[0].system_sum_count)

813


### ACTYCAP

In [44]:
# comm_date_bucket - yyyymmddhh - 2024120100
rows = session.execute(get_acty_cap_stmt, ['credit_card', start_date, end_date])
print(rows[0].system_sum_count)

7924


### PRTYCAP

In [45]:
# comm_date_bucket - yyyymmddhh - 2024120100
rows = session.execute(get_prty_cap_stmt, ['ctgy_grp_1', 'ctgy_1', start_date, end_date])
print(rows[0].system_sum_count)

1618


### CHANNELCAP

In [46]:
# comm_date_bucket - yyyymmddhh - 2024120100
rows = session.execute(get_channel_cap_stmt, ['email', start_date, end_date])
print(rows[0].system_sum_count)

8098
